**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
#import skvideo
#skvideo.setFFmpegPath('/usr/local/lib/python3.6/dist-packages/ffmpeg/')
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, Dropout, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The epsilon is essential to our needs as it allows us to tackle one major concern in reinforcement learning, **the exploration / exploitation trade off**. 
>
> Indeed, each action can be either selected randomly, or selected such as to maximize the reward in the next state. Although our aim is to maximize the final reward, if we only focus on immediate reward at each step -**exploitation**-, we might not be able to perform actions that would yield a larger reward in the long run.
>
> For this reason, we decide to set some of our actions to be random actions. This would allow us to **explore** the environment in a more thorough way. Here is where the **epsilon** comes into play. It is a rate that determines how often the next action will be selected at random, i.e. if a randomly generated number is < epsilon.
>
> The best value for **epsilon** should be dynamic, as the agent progressively gets to know better its environment. Therefore, the value for epsilon should decrease through time as we should increasingly exploit our knowledge of the environment.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        # Creating three dimensions for the image, for visual purposes
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> * **Board**: the board array is the grid -representing the island- on which our rat is going to move. It is used to store the placements of the positive and negative rewards, i.e. the cheese and the poison, here called 'bouns' and 'malus' respectively. Each epoch, we start a fresh game and reset the board and the positions of the cheese and poison. Their rewards are respectively of 0.5 and -1. When the rat lands on one of those positions, we set its value to 0.
>
> * **Position**: the position array also represents the island and the area on which our rat will move. However, this board is used to store the positions of the rat. Those positions are dynamic, as our rat moves through time, and are stored in the self.x and self.y coordinate variables. They change everytime the self.act function is called. The rat cannot go outside of the grid, and the contours of the grid -representing the sea- are here set to -1.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        # We are implementing here a random agent, therefore our direction is selected randomly
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [13]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))


In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/19.0. Average score (-12.5)
Win/lose count 8.5/15.0. Average score (-9.5)
Win/lose count 10.0/16.0. Average score (-8.333333333333334)
Win/lose count 8.5/13.0. Average score (-7.375)
Win/lose count 11.5/13.0. Average score (-6.2)
Win/lose count 8.0/15.0. Average score (-6.333333333333333)
Win/lose count 9.0/10.0. Average score (-5.571428571428571)
Win/lose count 12.5/17.0. Average score (-5.4375)
Win/lose count 11.5/14.0. Average score (-5.111111111111111)
Win/lose count 10.0/18.0. Average score (-5.4)
Final score: -5.4


> As expected, the average score is quite bad since the rat behaves randomly and does not learn to fetch the cheese.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




>First, we know that by definition
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> . \end{equation*}
>From there we obtain 
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{t=1}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> . \end{equation*}
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \gamma \sum_{t=0}^\infty\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \> . \end{equation*}
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)] + E_{p^{\pi}}[\gamma \sum_{t=0}^\infty\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \> . \end{equation*}
>And we get:
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)] + E_{p^{\pi}}[E_{(s',a')\sim p(.|s,a)}[\gamma \sum_{t=0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']] \> . \end{equation*}
>Indeed, $\gamma \sum_{t=0}^\infty\gamma^{t}r(s_{t+1},a_{t+1})$ only depends on the expectation to get values $s'$ and $a'$, depending on the initial values $s$ and $a$. 
>
>As it does not depend on $s'$ and $a'$, $$E_{p^{\pi}}[r(s,a)] = E_{(s',a')\sim p(.|s,a)}[r(s,a)]$$
>
>Therefore:
>\begin{equation*}Q^\pi(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)] + E_{(s',a')\sim p(.|s,a)}[E_{p^{\pi}}[\gamma \sum_{t=0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']] \> . \end{equation*}
>\begin{equation*}Q^\pi(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a) + E_{p^{\pi}}[\gamma \sum_{t=0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']] \> . \end{equation*}
>
>Finally, we get:
>\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
>
> ***
>
>Next, we know that $Q^*(s,a)=\max_{\pi}Q^\pi(s,a)$, showing that this is achieved by the optimal value of $\pi$, $\pi^*$.
> 
> We get:
\begin{equation*} Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] \end{equation*}
>
> Yet we know that $\pi^*(s)=arg \ \underset{a}{max} \ Q_\pi(s,a)$
>
> Therefore, $Q^{\pi^*}(s',a')=\max_{a'}Q^{*}(s',a')$ and we get:
\begin{equation*} Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] \end{equation*}
>
> ***
>
> Finally, one plausible objective for our loss function would we to minimize the distance between our training value function and the optimal value function that we have calculated in our model thanks to the previous question.
>
> We could use the squared norm to do so, therefore getting:
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [7]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            # We want to keep track of our most recent moves, which contain more valuable information.
            # We therefore erase the oldest value and append the new one
            self.memory = self.memory[1:]
            self.memory.append(m)
        pass

    def random_access(self):
        index = np.random.randint(len(self.memory))
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [8]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [9]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # The learned act corresponds to the one yielding the highest value, as predicted by our model
        predict = self.model.predict(np.expand_dims(s,0))
        return np.argmax(predict)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):

            state, next_state, action, reward, game_over = self.memory.random_access()
            # We will take from our memory pool a sample corresponding to the length of our batch size.
            # This will be our input state.
            input_states[i] = state.copy()
            
            if game_over_:
                # If the game is over, we can only get the reward, and cannot predict the next move.
                target_q[i, action] = reward
            else:
                # Otherwise, we use the formula obtained in question 5 to set our target, i.e. the maximal value function
                target_q[i, action] = reward + self.discount * max(self.model.predict(np.expand_dims(next_state,0))[0])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [37]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent, env, 200, prefix='fc_train')
#HTML(display_videos('fc_train10.mp4'))

Epoch 000/200 | Loss 0.0186 | Win/lose count 9.0/9.0 (0.0)
Epoch 001/200 | Loss 0.0262 | Win/lose count 7.5/8.0 (-0.5)
Epoch 002/200 | Loss 0.0121 | Win/lose count 5.0/8.0 (-3.0)
Epoch 003/200 | Loss 0.0159 | Win/lose count 9.5/4.0 (5.5)
Epoch 004/200 | Loss 0.0269 | Win/lose count 16.0/7.0 (9.0)
Epoch 005/200 | Loss 0.0279 | Win/lose count 7.0/4.0 (3.0)
Epoch 006/200 | Loss 0.0249 | Win/lose count 12.5/14.0 (-1.5)
Epoch 007/200 | Loss 0.0167 | Win/lose count 10.0/8.0 (2.0)
Epoch 008/200 | Loss 0.0041 | Win/lose count 7.5/4.0 (3.5)
Epoch 009/200 | Loss 0.0194 | Win/lose count 13.0/10.0 (3.0)
Epoch 010/200 | Loss 0.0235 | Win/lose count 10.0/5.0 (5.0)
Epoch 011/200 | Loss 0.0103 | Win/lose count 7.0/8.0 (-1.0)
Epoch 012/200 | Loss 0.0074 | Win/lose count 7.0/5.0 (2.0)
Epoch 013/200 | Loss 0.0263 | Win/lose count 13.0/12.0 (1.0)
Epoch 014/200 | Loss 0.0101 | Win/lose count 14.5/7.0 (7.5)
Epoch 015/200 | Loss 0.0145 | Win/lose count 11.5/8.0 (3.5)
Epoch 016/200 | Loss 0.0030 | Win/lose co

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [10]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()

        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(5,5,self.n_state)))
        model.add(Conv2D(16, (3, 3), activation='relu'))

        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [94]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0188 | Win/lose count 4.5/6.0 (-1.5)
Epoch 001/100 | Loss 0.0114 | Win/lose count 11.5/9.0 (2.5)
Epoch 002/100 | Loss 0.0100 | Win/lose count 9.5/3.0 (6.5)
Epoch 003/100 | Loss 0.0113 | Win/lose count 7.0/7.0 (0.0)
Epoch 004/100 | Loss 0.0122 | Win/lose count 11.0/6.0 (5.0)
Epoch 005/100 | Loss 0.0169 | Win/lose count 12.0/11.0 (1.0)
Epoch 006/100 | Loss 0.0238 | Win/lose count 14.0/12.0 (2.0)
Epoch 007/100 | Loss 0.0115 | Win/lose count 7.0/3.0 (4.0)
Epoch 008/100 | Loss 0.0357 | Win/lose count 9.5/5.0 (4.5)
Epoch 009/100 | Loss 0.0168 | Win/lose count 11.0/6.0 (5.0)
Epoch 010/100 | Loss 0.0119 | Win/lose count 11.5/4.0 (7.5)
Epoch 011/100 | Loss 0.0201 | Win/lose count 11.0/3.0 (8.0)
Epoch 012/100 | Loss 0.0136 | Win/lose count 15.0/12.0 (3.0)
Epoch 013/100 | Loss 0.0062 | Win/lose count 5.5/6.0 (-0.5)
Epoch 014/100 | Loss 0.0219 | Win/lose count 6.5/6.0 (0.5)
Epoch 015/100 | Loss 0.0107 | Win/lose count 14.0/7.0 (7.0)
Epoch 016/100 | Loss 0.0078 | Win/lose cou

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [38]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.5/1.0. Average score (6.5)
Win/lose count 2.0/0. Average score (4.25)
Win/lose count 5.0/3.0. Average score (3.5)
Win/lose count 4.0/0. Average score (3.625)
Win/lose count 16.0/2.0. Average score (5.7)
Win/lose count 4.0/0. Average score (5.416666666666667)
Win/lose count 3.5/0. Average score (5.142857142857143)
Win/lose count 2.0/0. Average score (4.75)
Win/lose count 5.0/0. Average score (4.777777777777778)
Win/lose count 7.0/0. Average score (5.0)
Win/lose count 2.5/0. Average score (4.7727272727272725)
Win/lose count 9.5/1.0. Average score (5.083333333333333)
Win/lose count 13.0/0. Average score (5.6923076923076925)
Win/lose count 6.0/1.0. Average score (5.642857142857143)
Win/lose count 2.0/0. Average score (5.4)
Win/lose count 4.0/0. Average score (5.3125)
Win/lose count 11.5/1.0. Average score (5.617647058823529)
Win/lose count 3.5/0. Average score (5.5)
Win/lose count 11.0/0. Average score (5.7894736842105265)
Win/lose count 5.0/0. Average scor

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

> Although both our models seem to converge during training, we can clearly see in the videos that the rats seem reluctant to explore the entire map, and sometimes get stuck in a small area which maximizes their immediate reward.
>
> During testing however, there are some clear differences. The CNN, which already yielded better results that the FC during training, has a high and positive final score, contrary to the FC. The FC model seems to overfit the training data, and to have learnt little information.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [33]:
def train_explore(agent,env,epoch,prefix=''):
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        
        # Possibility to change the epsilon
        #if e>0:
        #    agent.set_epsilon(0.5*(1- (e/epoch)))
            
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        # Creating three dimensions for the image, for visual purposes
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            # If the rat has already landed on the position, it get an incentive to explore other locations by accumulating a malus
            reward = -self.malus_position[self.x, self.y]
            
        reward = reward + self.board[self.x, self.y]
            
        self.board[self.x, self.y] = 0
        # After the rat reaches a specific position, this position gives a malus
        self.malus_position[self.x, self.y] = 0.01
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [34]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 200, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/200 | Loss 0.0027 | Win/lose count 5.0/5.719999999999978 (-0.7199999999999784)
Epoch 001/200 | Loss 0.0031 | Win/lose count 6.8900000000000015/4.759999999999985 (2.1300000000000168)
Epoch 002/200 | Loss 0.0074 | Win/lose count 4.45/6.809999999999964 (-2.359999999999964)
Epoch 003/200 | Loss 0.0155 | Win/lose count 4.5/2.7499999999999845 (1.7500000000000155)
Epoch 004/200 | Loss 0.0171 | Win/lose count 11.860000000000003/11.509999999999971 (0.3500000000000316)
Epoch 005/200 | Loss 0.0075 | Win/lose count 5.420000000000001/2.829999999999992 (2.5900000000000087)
Epoch 006/200 | Loss 0.0158 | Win/lose count 10.790000000000003/10.699999999999966 (0.09000000000003716)
Epoch 007/200 | Loss 0.0033 | Win/lose count 8.850000000000001/8.779999999999964 (0.07000000000003759)
Epoch 008/200 | Loss 0.0136 | Win/lose count 4.900000000000001/3.9099999999999815 (0.9900000000000198)
Epoch 009/200 | Loss 0.0206 | Win/lose count 10.780000000000003/6.789999999999984 (3.990000000000019)
Epoch 010/2

In [35]:
# Evaluation
test(agent,env,200,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 2.0/0. Average score (2.0)
Win/lose count 3.0/0. Average score (2.5)
Win/lose count 4.0/0. Average score (3.0)
Win/lose count 9.0/0. Average score (4.5)
Win/lose count 20.5/2.0. Average score (7.3)
Win/lose count 6.5/0. Average score (7.166666666666667)
Win/lose count 11.0/0. Average score (7.714285714285714)
Win/lose count 6.0/0. Average score (7.5)
Win/lose count 11.5/0. Average score (7.944444444444445)
Win/lose count 5.5/0. Average score (7.7)
Win/lose count 8.5/1.0. Average score (7.681818181818182)
Win/lose count 7.0/0. Average score (7.625)
Win/lose count 6.5/0. Average score (7.538461538461538)
Win/lose count 13.0/1.0. Average score (7.857142857142857)
Win/lose count 4.5/0. Average score (7.633333333333334)
Win/lose count 12.0/0. Average score (7.90625)
Win/lose count 5.5/0. Average score (7.764705882352941)
Win/lose count 1.5/0. Average score (7.416666666666667)
Win/lose count 14.5/1.0. Average score (7.7368421052631575)
Win/lose count 5.0/0. Average score (7.6)

> This model yields way better results. Indeed, the rat now gets an incentive to keep moving through the field, and avoids getting stuck at one specific place.
>
> Changing the epsilon through time, from 0.5 to 0, proportionally to the number of epochs, yielded results inferior to those obtained. I decided not to keep that option.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***